In [ ]:
# default_exp inference

# Inference

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export

import numpy as np
from fastai.basics import *

In [ ]:
#export

from vqa.wrangling import make_dataframe, label_dataframe, make_framer

def make_test_dataframe(root, frame_indices_list):
    return (
        make_dataframe(root)
        .pipe(label_dataframe)
        .pipe(make_framer(frame_indices_list))
    )

In [ ]:
#export

from scipy.stats import mode

most_freq = lambda seq: mode(seq)[0][0]

def decode_preds(vocabs, preds):
    for item_preds in zip(pred.cpu().numpy() for pred in preds):
        yield [vocab[p] for vocab, p in zip(vocabs, item_preds)]

def fill_preds(dataf, dls, preds):
    pred_distortion, pred_severity =  list(zip(*list(decode_preds(dls, preds))))
    dataf['distortion_preds'] = pred_distortion
    dataf['severity_preds'] = pred_severity
    return dataf

def aggregate_preds(dataf):
    idf = dataf.groupby(by='video_name').agg({
        'distortion': first, 
        'distortion_preds': list, 
        'severity': first, 
        'severity_preds': list, 
    })
    idf['distortion_inference'] = idf['distortion_preds'].apply(most_freq)
    idf['severity_inference'] = idf['severity_preds'].apply(most_freq)
    return idf

def format_pred(pred):
    distortions = pred[0].split("_")
    if len(distortions)==0:
        return ''
    severity = pred[1]
    return ','.join(sorted([f"{distortion[1]}_{severity[1]}" for distortion in distortions]))

def make_submission(distortion, severity):
    out = '\n'.join(L(zip(distortion, severity)).map(format_pred))
    with open('predict.txt', 'w') as f:
        f.write(out)
    return out